## step 4 - Building a Predictive Model for Hotel Pricing Dynamics Using Snapshot Data (V2)




- **Objective:**  
  Predict the `Discount Price` for check-in dates within 30 days of a given `Snapshot Date` while minimizing the number of price samples (queries) from the database.

- **Context:**  
  Given fields of `Snapshot Date`, `Hotel Name`, `Discount Code`, build a relevant data set to predict the `Discount Price` for check-in dates within 30 days of the `Snapshot Date`.

- **Performance Measure:**  
  The model's success is evaluated by the R-squared metric on a subset of 40 hotels with the most complete data.


### Solution Approaches: Gaussian Process Regression vs. Bayesian Linear and Polynomial Regression



#### Gaussian Process Regression



- **Pros:**
  - **Uncertainty Quantification:** Provides both predictions and confidence intervals, which is key for active sampling.
  - **Sample Efficiency:** Excels when data is limited, helping us select the most informative samples.
  - **Flexibility:** Capable of modeling complex, non-linear relationships inherent in price trends.
  
- **Cons:**
  - **Computational Cost:** Can become computationally expensive with larger datasets.
  - **Kernel Selection:** Requires careful tuning of the kernel functions to capture the underlying data patterns accurately.

- **Fit to the Problem:**  
  Ideal for our task since its uncertainty estimates allow us to strategically choose the next check-in dates to sample, ensuring we use as few queries as possible while maintaining high prediction performance.




#### Bayesian Linear and Polynomial Regression



- **Pros:**
  - **Simplicity:** Easy to implement and interpret.
  - **Speed:** Generally faster to train compared to GP, which can be advantageous with simpler or smooth trends.
  
- **Cons:**
  - **Limited Flexibility:** May not capture complex non-linear patterns as effectively.
  - **Uncertainty Estimates:** While Bayesian methods provide uncertainty measures, they are often less robust in capturing the variability in complex datasets compared to GP.

- **Fit to the Problem:**  
  Suitable if the price trends were very smooth and predictable. However, the nuances in hotel pricing (e.g., varying discount strategies, day-of-week effects) suggest a need for a more flexible approach.




#### Chosen Approach:



- **Gaussian Process Regression** is selected as the primary method. Its ability to provide uncertainty estimates and guide active sampling makes it the best fit for minimizing the number of price queries while still achieving a high R-squared on the test set.


### Data Loading and Extraction:

1. **Load the Data:**  
   We read the CSV file into a DataFrame and ensure that date columns are properly converted to datetime objects.

2. **Filter the Data:**  
   We filter the dataset to include only rows where `DayDiff` is 30 or less, meaning the check-in is within 30 days after the snapshot date.

3. **Group the Data:**  
   We group by `Snapshot Date`, `Hotel Name`, and `Discount Code`, and count the number of transactions (rows) in each group. This count is stored in the column `transaction_count`.

4. **Dataset: onePerHotel:**  
   - For each hotel, we sort by `transaction_count` in descending order and select the combination with the maximum count.  
   - Then, we sort these unique hotel rows by `transaction_count` and take the top 40.

5. **Dataset: mostData:**  
   - We simply sort all the grouped combinations by `transaction_count` in descending order and select the top 40 combinations.

6. **Extracting Parameters:**  
   For each dataset, we extract a list of dictionaries (or "params") containing `Snapshot Date`, `Hotel Name`, and `Discount Code`.


In [ ]:
import pandas as pd

data_path = "./hotels_data_changed.csv"
data = pd.read_csv(data_path)

# We are only interested in transactions with a check-in date within 30 days after the snapshot.
data_filtered = data[data['DayDiff'] <= 30].copy()

data_filtered['Snapshot Date'] = pd.to_datetime(data_filtered['Snapshot Date']).dt.normalize()

# We group by the combination of 'Snapshot Date', 'Hotel Name', and 'Discount Code', and count the number of transactions.
grouped = data_filtered.groupby(
    ['Snapshot Date', 'Hotel Name', 'Discount Code']
).size().reset_index(name='transaction_count')

# For each hotel, choose the combination with the highest transaction_count.
# First, sort within each hotel so that the highest count is on top,
# then group by 'Hotel Name' and take the first (best) row.
onePerHotel = (
    grouped.sort_values(['Hotel Name', 'transaction_count'], ascending=[True, False])
    .groupby('Hotel Name')
    .first()
    .reset_index()
)

onePerHotel = onePerHotel.sort_values('transaction_count', ascending=False).head(40)

mostData = grouped.sort_values('transaction_count', ascending=False).head(40)

params_onePerHotel = onePerHotel[['Snapshot Date', 'Hotel Name', 'Discount Code']].to_dict('records')
params_mostData   = mostData[['Snapshot Date', 'Hotel Name', 'Discount Code']].to_dict('records')

print("onePerHotel (Top 40 hotels with best combination per hotel):")
display(onePerHotel.head())

print("mostData (Top 40 combinations overall):")
display(mostData.head())

### Feature Engineering and Data Preparation 

In this section we would create a function that gets the `Snapshot Date`, `Hotel Name`, and `Discount Code` and returns all we need to start training the model.
This includes the following:
-  get relevant data from the dataset
-  apply feature engineering on that data
-  normalize and scale the data
-  Splitting into X and y & Creating the Wrapper Function

#### Get the data 

In [ ]:
def load_relevant_data(params, data):
 
    snapshot_date = pd.to_datetime(params['Snapshot Date']).normalize()
    
    subset = data[
        (data['Snapshot Date'] == snapshot_date) &
        (data['Hotel Name'] == params['Hotel Name']) &
        (data['Discount Code'] == params['Discount Code']) &
        (data['DayDiff'] <= 30) 
    ]
    return subset

test_params = params_mostData[0]
relevant_subset = load_relevant_data(test_params, data_filtered)
print("Parameters:", test_params)
print("Relevant Data (first 5 rows):")
display(relevant_subset.head())

#### Feature engineering

In this step, we transform our raw data into a format that is ready for model training. The goals are to:

- **Convert Date Information:**  
  We convert the date column `Checkin Date` into numerical values (Unix timestamps) so that the model can understand and work with temporal data.

- **Encode Weekday Information:**  
  We convert the weekday from a string (e.g., "Monday") into a numeric value using the helper function `convert_weekday_to_num`, storing it as `WeekDay_num`.

- **Remove Irrelevant or Constant Columns:**  
  After feature engineering, some columns become redundant or uninformative:
  - **`Hotel Name`:** Removed because the data is filtered for a specific hotel, or if used for filtering, the raw name is no longer needed.
  - **`Checkin Date`:** Once their numerical representations are extracted, the raw date values are no longer required.
  - **`WeekDay`:** We already have the numeric `WeekDay_num`, making the original string column redundant.
  - **`Days`:** This column is always the same (always 5), so it does not provide any variance or useful information for training.
  - **`Snapshot Date`:** Since the snapshot is the same across the subset, this column is constant and does not help differentiate between records.
  - **`Snapshot ID`:** This is also constant for a given snapshot and can be removed to reduce noise.

The removal of these columns helps to simplify our dataset and ensures that the model is trained only on features that provide meaningful variation.


In [ ]:
def convert_weekday_to_num(weekday):
    mapping = {
        'Monday': 0,
        'Tuesday': 1,
        'Wednesday': 2,
        'Thursday': 3,
        'Friday': 4,
        'Saturday': 5,
        'Sunday': 6
    }
    return mapping.get(weekday, -1)  # Returns -1 if the weekday is not found

def feature_engineering(df):
    df['Checkin Date'] = pd.to_datetime(df['Checkin Date'])
    
    df['Checkin_Date_num'] = df['Checkin Date'].apply(lambda x: x.timestamp())
    
    
    df['WeekDay_num'] = df['WeekDay'].apply(convert_weekday_to_num)
    
    # Remove columns that are not useful for training:
    # 'Hotel Name'   - No longer needed since the data is already filtered per hotel.
    # 'Checkin Date' - Raw dates are replaced by numerical features.
    # 'WeekDay'      - Replaced by 'WeekDay_num'.
    # 'Days'         - Always constant (5) across the dataset.
    # 'Snapshot Date' - Constant within a given snapshot.
    # 'Snapshot ID'  - Constant within the dataset.
    columns_to_drop = ['Hotel Name', 'Snapshot Date', 'Checkin Date', 'WeekDay', 'Days', 'Snapshot ID' ]
    df = df.drop(columns=columns_to_drop)
    
    return df
engineered_data = feature_engineering(relevant_subset.copy())

print("Engineered Data (first 5 rows):")
display(engineered_data.head())
print(engineered_data.columns)

#### Normalize and scale the data

**What are we doing?**  
We use a `StandardScaler` to standardize the continuous features by removing the mean and scaling to unit variance. This process prevents features with larger numeric ranges from dominating the learning process.

**Why are we doing it?**  
Normalization is crucial in many machine learning algorithms—especially when using distance-based metrics or gradient descent—for faster convergence and improved performance.

**Features to Scale:**  
Based on our engineered dataset, we scale:
- `Checkin_Date_num` (numerical representation of the check-in date)  
- `Original Price`  
- `Discount Price`  
- `Available Rooms`  
- `DayDiff`  
- `DiscountDiff`  
- `DiscountPerc`

In [ ]:
from sklearn.preprocessing import StandardScaler

def normalize_features(df, features_to_scale, scaler=None):
    if scaler is None:
        scaler = StandardScaler()
    
    df_scaled = df.copy()
    
    df_scaled[features_to_scale] = scaler.fit_transform(df_scaled[features_to_scale])
    
    return df_scaled, scaler

features_to_scale = [
    "Checkin_Date_num", 
    "Original Price", 
    "Discount Price", 
    "Available Rooms", 
    "DayDiff", 
    "DiscountDiff", 
    "DiscountPerc"
]

normalized_data, fitted_scaler = normalize_features(engineered_data, features_to_scale)

print("Normalized Data (first 5 rows):")
display(normalized_data.head())

####  Splitting into X and y & Creating the Wrapper Function

In this step, we:

- Split the normalized data into features (`X`) and target (`y`). Here, `y` is the scaled "Discount Price".  
- Create a wrapper function that runs the entire preprocessing pipeline (from getting relevant data to normalization) and returns `X`, `y`, the fitted scaler, and a helper function to reverse the scaling for the target.  
- The helper function uses the scaling parameters for "Discount Price" (found in the fitted scaler) so that later we can convert predictions back to their original values.

> **Why Remove "Discount Price" from X?**  
> The target variable should not be present in the features. By dropping "Discount Price" from the normalized data, we ensure that `X` contains only the input features while `y` contains the target variable.

> **Reversing the Scaling:**  
> Since the scaler is fitted on multiple columns, we locate the index corresponding to "Discount Price" in our `features_to_scale` list and use its mean and scale to convert scaled predictions back to their original values.

In [ ]:
X.head()

In [ ]:
def prepare_training_data(params, data, sort=False):
    relevant = load_relevant_data(params, data)
    engineered = feature_engineering(relevant.copy())
    normalized, scaler = normalize_features(engineered, features_to_scale)
    price_related_columns = ["Discount Price","DiscountPerc", "DiscountDiff", "Original Price"]
    if sorted:
        normalized = normalized.sort_values(by="Discount Price")

    X = normalized.drop(columns=price_related_columns)
    y = normalized["Discount Price"]

    idx = features_to_scale.index("Discount Price")
    
    def reverse_scaling(y_scaled):
        return y_scaled * scaler.scale_[idx] + scaler.mean_[idx]
    
    return X, y, scaler, reverse_scaling

X, y, scaler, reverse_scaling = prepare_training_data(test_params,data_filtered )
print("X shape:", X.shape)
print("y shape:", y.shape)
print("First 5 values of y (scaled):", y.head().values)
print("First 5 values of y (original):", reverse_scaling(y.head().values))


### Gaussian Process Regression: Training And Testing



#### Model Initialization



In this cell, we initialize our GaussianProcessRegressor model. We use a kernel that combines a ConstantKernel and an RBF kernel. 

- **GaussianProcessRegressor:**  
  Provides both predictions and uncertainty estimates, which are essential for our active sampling approach.

- **ConstantKernel:**  
  Represents a constant bias in the function, serving as a baseline.

- **RBF Kernel (Radial Basis Function):**  
  Models smooth variations in the data, which is ideal for capturing trends in discount prices.



In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def initialize_model():
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-2, normalize_y=True)
    return gp

model = initialize_model()
print("Initialized Model:")
print(model)


#### Active Sampling Loop


##### Maximum uncertainty based stop


In this cell, we implement the active sampling loop. The process is as follows:

- Start with an initial set of samples (e.g., the first, middle, and last indices).
- Iteratively fit the Gaussian Process model on the current training set.
- Predict on the remaining (unsampled) data and obtain uncertainty estimates.
- Select the candidate with the highest uncertainty and add it to the training set.
- Compute the `R²` score to monitor performance.
- Stop when the maximum uncertainty is below a threshold or when the maximum iterations are reached.



In [ ]:
import numpy as np
from sklearn.metrics import r2_score

def active_sampling_loop(X, y, max_iterations=10, uncertainty_threshold=0.05, initial_sample_indices=None):
    n_samples = X.shape[0]
    
    if initial_sample_indices is None:
        initial_sample_indices = [0, n_samples // 2, n_samples - 1]
    
    training_indices = set(initial_sample_indices)
    candidate_indices = set(range(n_samples)) - training_indices
    iteration_log = []
    
    for iteration in range(max_iterations):
        # Prepare current training data
        X_train = X.iloc[sorted(list(training_indices))]
        y_train = y.iloc[sorted(list(training_indices))]
        
        # Initialize and fit the Gaussian Process model
        model = initialize_model()
        model.fit(X_train, y_train)
        
        # Predict on candidate data and obtain uncertainties
        candidate_list = sorted(list(candidate_indices))
        if not candidate_list:
            break

        X_candidates = X.iloc[candidate_list]
        y_pred_candidates, std_candidates = model.predict(X_candidates, return_std=True)
        
        # Find candidate with maximum uncertainty
        max_std = np.max(std_candidates)
        chosen_candidate_idx = candidate_list[np.argmax(std_candidates)]
        
        y_pred_full = model.predict(X)
        current_r2 = r2_score(y, y_pred_full)
        iteration_log.append({
            'iteration': iteration,
            'num_samples': len(training_indices),
            'max_uncertainty': max_std,
            'r2': current_r2
        })
        
        
        if max_std < uncertainty_threshold:
            break
        
        # Add the most uncertain candidate to the training set
        training_indices.add(chosen_candidate_idx)
        candidate_indices.remove(chosen_candidate_idx)
    
    # Final training on the selected samples
    X_final = X.iloc[sorted(list(training_indices))]
    y_final = y.iloc[sorted(list(training_indices))]
    final_model = initialize_model()
    final_model.fit(X_final, y_final)
    final_predictions = final_model.predict(X)
    final_r2 = r2_score(y, final_predictions)
    
    return {
        'final_model': final_model,
        'final_predictions': final_predictions,
        'final_r2': final_r2,
        'iteration_log': iteration_log,
        'total_samples_used': len(training_indices)
    }

results = active_sampling_loop(X, y, max_iterations=15, uncertainty_threshold=0.05)

print("Final R² Score:", results['final_r2'])
print("Total Samples Used:", results['total_samples_used'])
print("Iteration Log:")
for log in results['iteration_log']:
    print(log)


##### Maximum uncertainty and stagnant Iterations based stop

In this cell we implement an enhanced active sampling loop. In addition to stopping when the maximum uncertainty is very low, we also monitor `R²` improvement. The loop will stop if either:
- The model's uncertainty is very low (`max_std < uncertainty_threshold`), or
- The uncertainty is moderately low (`max_std < higher_uncertainty_threshold`) **and** the `R²` improvement has been stagnant for a set number of iterations.


In [ ]:
def active_sampling_loop_with_stagnant_iterations_stop(X, y, max_iterations=10, 
                                                        uncertainty_threshold=0.05, 
                                                        higher_uncertainty_threshold=0.1,
                                                        initial_sample_indices=None, 
                                                        r2_improvement_threshold=1e-4, 
                                                        max_stagnant_iterations=3):
    n_samples = X.shape[0]
    if initial_sample_indices is None:
        initial_sample_indices = [0, n_samples // 2, n_samples - 1]
    
    training_indices = set(initial_sample_indices)
    candidate_indices = set(range(n_samples)) - training_indices
    iteration_log = []
    
    previous_r2 = -np.inf  
    stagnant_iterations = 0  
    
    for iteration in range(max_iterations):
        # Prepare training data for the current iteration
        X_train = X.iloc[sorted(list(training_indices))]
        y_train = y.iloc[sorted(list(training_indices))]
        
        # Initialize and fit the model
        model = initialize_model()
        model.fit(X_train, y_train)
        
        # Predict on candidate data to obtain uncertainties
        candidate_list = sorted(list(candidate_indices))
        if not candidate_list:
            break

        X_candidates = X.iloc[candidate_list]
        y_pred_candidates, std_candidates = model.predict(X_candidates, return_std=True)
        
        max_std = np.max(std_candidates)
        chosen_candidate_idx = candidate_list[np.argmax(std_candidates)]
        
        
        y_pred_full = model.predict(X)
        current_r2 = r2_score(y, y_pred_full)
        
        
        r2_improvement = current_r2 - previous_r2
        if r2_improvement < r2_improvement_threshold:
            stagnant_iterations += 1
        else:
            stagnant_iterations = 0
        
        previous_r2 = current_r2  
        
        iteration_log.append({
            'iteration': iteration,
            'num_samples': len(training_indices),
            'max_uncertainty': max_std,
            'r2': current_r2,
            'r2_improvement': r2_improvement
        })
        
        
        # Stop if the model is very confident OR if it is moderately confident and not improving.
        if (max_std < uncertainty_threshold) or ((max_std < higher_uncertainty_threshold) and (stagnant_iterations >= max_stagnant_iterations)):
            break
        
        # Add the candidate with highest uncertainty to the training set
        training_indices.add(chosen_candidate_idx)
        candidate_indices.remove(chosen_candidate_idx)
    
    # Final training on the selected samples
    X_final = X.iloc[sorted(list(training_indices))]
    y_final = y.iloc[sorted(list(training_indices))]
    final_model = initialize_model()
    final_model.fit(X_final, y_final)
    final_predictions = final_model.predict(X)
    final_r2 = r2_score(y, final_predictions)
    
    return {
        'final_model': final_model,
        'final_predictions': final_predictions,
        'final_r2': final_r2,
        'iteration_log': iteration_log,
        'total_samples_used': len(training_indices)
    }


results = active_sampling_loop_with_stagnant_iterations_stop(X, y, max_iterations=15, 
                                                              uncertainty_threshold=0.05, 
                                                              higher_uncertainty_threshold=0.1,
                                                              r2_improvement_threshold=1e-4, 
                                                              max_stagnant_iterations=3)

print("Final R² Score:", results['final_r2'])
print("Total Samples Used:", results['total_samples_used'])
print("Iteration Log:")
for log in results['iteration_log']:
    print(log)


#### Visualizing the Results



In this cell we visualize the active sampling process. The first part prints a log that shows, for each iteration, the number of samples used, the maximum uncertainty, and the R² score. The chart below shows two plots:
- **R² Evolution:** How the model's performance improves over iterations.
- **Max Uncertainty Evolution:** How the maximum prediction uncertainty decreases as more samples are added.


In [ ]:
import matplotlib.pyplot as plt

def print_iteration_log(results, total_data_count):
    final_samples = results['total_samples_used']
    percent_used = final_samples / total_data_count * 100
    print(f"Total Data Available: {total_data_count}")
    print(f"Total Samples Used: {final_samples} ({percent_used:.2f}%)")
    print("\nIteration Log:")
    for log in results['iteration_log']:
        print(f"Iteration {log['iteration']}: {log['num_samples']} samples, "
              f"Max Uncertainty: {log['max_uncertainty']:.4f}, R²: {log['r2']:.4f}")
    print(f"\nFinal R² Score: {results['final_r2']:.4f}")

def plot_active_sampling_results(results, total_data_count):
    iterations = [log['iteration'] for log in results['iteration_log']]
    r2_values = [log['r2'] for log in results['iteration_log']]
    uncertainties = [log['max_uncertainty'] for log in results['iteration_log']]
    
    final_samples = results['total_samples_used']
    percent_used = final_samples / total_data_count * 100

    fig, ax = plt.subplots(2, 1, figsize=(8, 10))
    
    ax[0].plot(iterations, r2_values, marker='o', linestyle='-')
    ax[0].set_title('R² Evolution Over Iterations')
    ax[0].set_xlabel('Iteration')
    ax[0].set_ylabel('R² Score')
    ax[0].grid(True)
    ax[0].text(0.5, 0.1, f'Final R²: {results["final_r2"]:.4f}', 
               transform=ax[0].transAxes, fontsize=12, color='green',
               bbox=dict(facecolor='white', alpha=0.8, edgecolor='green'))
    
    
    ax[1].plot(iterations, uncertainties, marker='o', linestyle='-')
    ax[1].set_title('Max Uncertainty Evolution Over Iterations')
    ax[1].set_xlabel('Iteration')
    ax[1].set_ylabel('Max Uncertainty (std)')
    ax[1].grid(True)
    
    plt.suptitle(f'Active Sampling: {final_samples} of {total_data_count} samples used ({percent_used:.2f}%)', fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

total_data_count = X.shape[0]  # For example, if X has 44 rows

print_iteration_log(results, total_data_count)
plot_active_sampling_results(results, total_data_count)


### Gaussian Process Regression - Full dataset Testing

#### Active Sampling Model Execution

In this section, we execute our active sampling models over all parameter combinations. We provide two variants:
- **Basic Active Sampling Loop:** The standard loop that iteratively selects samples based on maximum uncertainty.
- **Active Sampling Loop with Stagnant Iterations Stop:** An enhanced loop that also monitors `R²` improvements and stops if the model's performance stagnates while uncertainty is moderately low.

This section prepares the results for each parameter combination by running the appropriate active sampling function and collecting key metrics such as the final R², total samples used, and iteration logs.


In [ ]:
%%capture
%pip install tqdm

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
from tqdm import tqdm

def run_active_sampling_basic(params_list, data, sort=False, verbose=False, **hyperparams):
    results_list = []
    if not verbose:
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    for i, param in enumerate(tqdm(params_list, desc="Running basic active sampling", disable=verbose)):
        X, y, scaler, reverse_scaling = prepare_training_data(param, data, sort=sort)
        total_data_count = X.shape[0] 
        
        result = active_sampling_loop(X, y, **hyperparams)
        result['params'] = param
        result['total_data_count'] = total_data_count
        results_list.append(result)
        if verbose:
            print(f"Completed parameter {i+1}/{len(params_list)}: R² = {result['final_r2']:.4f}, Samples = {result['total_samples_used']} of {X.shape[0]}")
            
    warnings.filterwarnings("default", category=ConvergenceWarning)

    return results_list

basic_hyperparams = {
    "max_iterations": 15,
    "uncertainty_threshold": 0.05,
}

results_basic = run_active_sampling_basic(
params_mostData[:3], data_filtered, 
verbose=True,
**basic_hyperparams
)

In [ ]:

def run_active_sampling_stagnant(params_list, data, sort=False ,verbose=False, **hyperparams):
    results_list = []
    if not verbose:
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    for i, param in enumerate(tqdm(params_list, desc="Running stagnant active sampling", disable=verbose)):
        X, y, scaler, reverse_scaling = prepare_training_data(param, data, sort=sort)
        total_data_count = X.shape[0] 
        
        result = active_sampling_loop_with_stagnant_iterations_stop(X, y, **hyperparams)
        result['params'] = param
        result['total_data_count'] = total_data_count 
        results_list.append(result)
        if verbose:
            print(f"Completed parameter {i+1}/{len(params_list)}: R² = {result['final_r2']:.4f}, Samples = {result['total_samples_used']} of {total_data_count}")
    
    warnings.filterwarnings("default", category=ConvergenceWarning)

    return results_list

stagnant_hyperparams = {
    "max_iterations": 15,
    "uncertainty_threshold": 0.05,
    "higher_uncertainty_threshold": 0.1,
    "r2_improvement_threshold": 1e-4,
    "max_stagnant_iterations": 3
}

results_stagnant = run_active_sampling_stagnant(
params_mostData[:3], data_filtered,
verbose =True,
**stagnant_hyperparams
)

#### Results Visualization & Preliminary Conclusions



##### Visualization 


In this cell, we visualize the aggregated results. The visualization function now accepts:
- `loop_type`: A string indicating the active sampling variant (e.g., "Basic Active Loop" or "Stagnant Active Loop").
- `hyperparams`: A dictionary of hyperparameter values used for the active sampling loop.

The function then creates a title that includes these hyperparameter values and displays three bar charts:
- Final `R²` scores.
- Total samples used.
- Data utilization percentages (samples used as a percentage of total available data).



In [ ]:
import matplotlib.pyplot as plt

def visualize_aggregated_results(results_list, loop_type, hyperparams):
    title_str = f"{loop_type} Results"
    
    hyperparams_items = [f"{k}: {v}" for k, v in hyperparams.items()]
    hyperparams_str = ""
    for i in range(0, len(hyperparams_items), 3):
        chunk = " | ".join(hyperparams_items[i:i+3])
        hyperparams_str += chunk + ("\n" if i + 3 < len(hyperparams_items) else "")
    
    final_r2_list = [r['final_r2'] for r in results_list]
    samples_used_list = [r['total_samples_used'] for r in results_list]
    total_data_counts = [r['total_data_count'] for r in results_list]
    data_utilization_list = [ (used / total) * 100 for used, total in zip(samples_used_list, total_data_counts)]
    
    n = len(results_list)
    indices = range(n)
    
    avg_r2 = sum(final_r2_list) / n
    avg_samples = sum(samples_used_list) / n
    avg_utilization = sum(data_utilization_list) / n
    
    # Create subplots for the aggregated metrics.
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Bar chart for final R².
    axes[0].bar(indices, final_r2_list, color='skyblue')
    axes[0].axhline(avg_r2, color='red', linestyle='--', label=f'Avg R² = {avg_r2:.4f}')
    axes[0].set_xlabel('Parameter Combination Index')
    axes[0].set_ylabel('Final R²')
    axes[0].set_title('Final R² for Each Combination')
    axes[0].legend()
    
    # Bar chart for total samples used.
    axes[1].bar(indices, samples_used_list, color='lightgreen')
    axes[1].axhline(avg_samples, color='red', linestyle='--', label=f'Avg Samples = {avg_samples:.2f}')
    axes[1].set_xlabel('Parameter Combination Index')
    axes[1].set_ylabel('Total Samples Used')
    axes[1].set_title('Samples Used per Combination')
    axes[1].legend()
    
    # Bar chart for data utilization (%).
    axes[2].bar(indices, data_utilization_list, color='plum')
    axes[2].axhline(avg_utilization, color='red', linestyle='--', label=f'Avg Utilization = {avg_utilization:.2f}%')
    axes[2].set_xlabel('Parameter Combination Index')
    axes[2].set_ylabel('Data Utilization (%)')
    axes[2].set_title('Data Utilization per Combination')
    axes[2].legend()
    
    # Set a prominent super-title at the top.
    plt.suptitle(title_str, fontsize=20, fontweight='bold', color='navy', y=0.98)
    
    # Add a centered hyperparameters container below the title.
    # This container is placed at y=0.93 and is horizontally centered (x=0.5).
    plt.gcf().text(0.5, 0.85, f"Hyperparameters:\n\n{hyperparams_str}", 
                   fontsize=12, ha='center', va='center',
                   bbox=dict(facecolor='lightgrey', alpha=0.6, boxstyle='round,pad=0.5'))
    
    # Adjust the layout so that subplots start below the hyperparameters container.
    plt.tight_layout(rect=[0, 0, 1, 0.83])
    plt.show()
    
    print(f"Average Final R²: {avg_r2:.4f}")
    print(f"Average Samples Used: {avg_samples:.2f}")
    print(f"Average Data Utilization: {avg_utilization:.2f}%")

In [ ]:
X.head()

In [ ]:
results_basic = run_active_sampling_basic(params_mostData, data_filtered, verbose=False, **basic_hyperparams)
visualize_aggregated_results(results_basic, "Basic Active Loop", basic_hyperparams)

In [ ]:
results_stagnant = run_active_sampling_stagnant(params_mostData, data_filtered, verbose=False, **stagnant_hyperparams)
visualize_aggregated_results(results_stagnant, "Stagnant Active Loop", stagnant_hyperparams)

##### Preliminary Conclusions

**Results**:

![Basic results updated](./images/step-4-Basic-initial-results.png)

![Stagnant results](./images/step-4-stagnat-initial-results.png)  

**Basic V.S. Stagnant**

- **Basic Active Sampling Loop:**
  - **Robustness:** Tends to use around 18 samples per parameter combination, yielding high `R²` scores (often 1.0000).
  - **Efficiency:** Uses more data, which may be unnecessary in cases with abundant information.

- **Stagnant Active Sampling Loop:**
  - **Efficiency:** Stops early when `R²` improvement stagnates, generally using fewer samples (between 11–16) and still achieving high performance in most cases.
  - **Risks:** In some cases (e.g., parameter combinations with very few total data points), the loop stops after only 3 samples, leading to negative `R²` values and clear underfitting.

 **Outliers and Data Issues**

- **Outlier Cases:**  
  Some parameter combinations consistently use only 3 samples and report poor performance (e.g., negative `R²`). These cases likely represent scenarios where the available data is sparse or noisy, right now its looking like  the most promising direction to investigate and improve.

- **Implication:**  
  Relying solely on the stagnant loop's stopping criterion can result in premature termination. A minimum sample threshold may be necessary to ensure that the model has enough data to learn meaningful patterns.

 **Next Steps**

1. **Revisit the Data:**  
   Analyze the outlier parameter combinations with low total data counts. Determine if these cases should be treated differently or if more data can be acquired.

2. **Implement a Minimum Sample Threshold:**  
   Adjust the stagnant loop to enforce a minimum number of samples (e.g., at least 5 or 7 samples) before allowing early termination (probably most promising result).

3. **Hyperparameter Tuning:**  
   Experiment with different settings for the uncertainty thresholds and maximum stagnant iterations to balance efficiency and robustness. Fine-tuning may reduce instances where the model stops too early.


### Final Optimization and Data Revision

In this section, would try to improve our results and address the points raised in the [preliminary conclusions](#preliminary-conclusions)

- Revisit the Data
- Implement a Minimum Sample Threshold
- Hyperparameter Tuning

#### Revisit the data

##### Revisit Parameter Sampling


Here we re-calculate the parameter combinations using our two strategies:
- **onePerHotel:** For each hotel, we select the combination (of `Snapshot Date`, `Hotel Name`, and `Discount Code`) with the highest transaction count.
- **mostData:** We select the top 40 overall combinations based on transaction counts.

We then display these results in a table (including the transaction counts) so we can review the distribution and identify any potential outliers in data availability.


In [ ]:
grouped = data_filtered.groupby(
    ['Snapshot Date', 'Hotel Name', 'Discount Code']
).size().reset_index(name='transaction_count')

onePerHotel = (
    grouped.sort_values(['Hotel Name', 'transaction_count'], ascending=[True, False])
    .groupby('Hotel Name')
    .first()
    .reset_index()
)
onePerHotel = onePerHotel.sort_values('transaction_count', ascending=False).head(40)

mostData = grouped.sort_values('transaction_count', ascending=False).head(40)

print("One Per Hotel (Top 40) Parameter Combinations with Transaction Counts:")
display(onePerHotel)

print("\nMost Data (Top 40) Parameter Combinations with Transaction Counts:")
display(mostData)


This looks solid to me, shows again the for maximum data we rather stick to Most Data params instead of One Per Hotel

##### Investigating Valid Data and Outliers for Parameter Combinations

Here, we further analyze the data returned by `load_relevant_data` for each parameter combination. For each combination, we compute:
- Total rows (number of transactions)
- Unique counts for WeekDays, Snapshot IDs, Check-in Dates, and Available Rooms
- Weekend count and Holiday-Connected count (using US holidays for 2014–2016, with a check for a holiday occurring within 3 days before or 1 day after the check-in date)
- Data range (from snapshot date to snapshot date + 30 days)
- **Price Metrics:**
  - Minimum and maximum Discount Price
  - Price range as a percentage
  - An array of price points (each price point is `[price, showcount]`)

We then split the parameter combinations into two groups (Good Predictions vs. Suspected Outliers) based on the final `R²` value and display an average comparison table.

In [ ]:
%%capture
%pip install holidays

In [ ]:
import pandas as pd
import holidays

# Create a US holidays object for the years 2014-2016.
us_holidays = holidays.US(years=[2014, 2015, 2016])

def is_connected_to_holiday(checkin_date, window_before=3, window_after=1):
    checkin_date = pd.to_datetime(checkin_date)
    start_window = checkin_date - pd.Timedelta(days=window_before)
    end_window = checkin_date + pd.Timedelta(days=window_after)
    for holiday_date in us_holidays.keys():
        holiday_ts = pd.to_datetime(holiday_date)
        if start_window <= holiday_ts <= end_window:
            return True
    return False

summary_metrics = []
for result in results_stagnant:  # (Use results from the stagnant or basic loop as needed)
    param = result['params']
    df_subset = load_relevant_data(param, data_filtered)
    total_rows = df_subset.shape[0]
    
    unique_weekdays = df_subset['WeekDay'].nunique() if 'WeekDay' in df_subset.columns else None
    unique_snapshot_ids = df_subset['Snapshot ID'].nunique() if 'Snapshot ID' in df_subset.columns else None
    unique_checkin_dates = df_subset['Checkin Date'].nunique() if 'Checkin Date' in df_subset.columns else None
    unique_rooms = df_subset['Available Rooms'].nunique() if 'Available Rooms' in df_subset.columns else None
    
    weekend_count = df_subset[df_subset['WeekDay'].isin(["Friday", "Saturday", "Sunday"])].shape[0]
    holiday_connected_count = df_subset[df_subset['Checkin Date'].apply(is_connected_to_holiday)].shape[0]
    
    snapshot_dt = pd.to_datetime(param['Snapshot Date'])
    data_range = f"{snapshot_dt.strftime('%Y-%m-%d')} - {(snapshot_dt + pd.Timedelta(days=30)).strftime('%Y-%m-%d')}"
    
    # Price metrics
    if total_rows > 0:
        discount_price_min = df_subset["Discount Price"].min()
        discount_price_max = df_subset["Discount Price"].max()
        discount_price_mean = df_subset["Discount Price"].mean()
        discount_price_std = df_subset["Discount Price"].std()
        price_range_percent = ((discount_price_max - discount_price_min) / discount_price_min) * 100 if discount_price_min != 0 else None
        price_counts = df_subset["Discount Price"].value_counts().sort_index()
        price_points = [[price, count] for price, count in price_counts.items()]
    else:
        discount_price_min = discount_price_max = discount_price_mean = discount_price_std = price_range_percent = None
        price_points = []
    
    param_key = f"{param['Snapshot Date']} | {param['Hotel Name']} | {param['Discount Code']}"
    
    summary_metrics.append({
        "Parameter": param_key,
        "Final R²": result['final_r2'],
        "Total Rows": total_rows,
        "Unique WeekDays": unique_weekdays,
        "Unique Snapshot IDs": unique_snapshot_ids,
        "Unique Checkin Dates": unique_checkin_dates,
        "Unique Rooms": unique_rooms,
        "Weekend Count": weekend_count,
        "Holiday Connected Count": holiday_connected_count,
        "Data Range": data_range,
        "Price Points": price_points,
        "Price Points length": len(price_points),
    })

df_metrics = pd.DataFrame(summary_metrics)

# Split into groups based on final R² (good predictions vs. suspected outliers)
df_predicting_well = df_metrics[df_metrics["Final R²"] >= 0]
df_outliers = df_metrics[df_metrics["Final R²"] < 0]

# Compute averages for numeric columns (price metrics will be averaged only for percentage metrics)
avg_good = df_predicting_well.mean(numeric_only=True)
avg_bad = df_outliers.mean(numeric_only=True)

avg_table = pd.DataFrame({"Good Predictions": avg_good, "Bad Predictions": avg_bad})
print("Average Metrics Comparison:")
display(avg_table)

print("\nDetailed Metrics for Good Predictions:")
display(df_predicting_well)
print("\nDetailed Metrics for Suspected Outliers:")
display(df_outliers)


 **Average Metrics Comparison**

| Metric                      | Good Predictions | Bad Predictions  |
|-----------------------------|------------------|------------------|
| Final R²                    | 0.999862         | -0.603804        |
| Total Rows                  | 27.677419        | 24.888889        |
| Unique WeekDays             | 6.419355         | 7.000000         |
| Unique Snapshot IDs         | 1.354839         | 1.111111         |
| Unique Checkin Dates        | 22.580645        | 23.666667        |
| Unique Rooms                | 5.225806         | 9.333333         |
| Weekend Count               | 9.741935         | 11.000000        |
| Holiday Connected Count     | 2.548387         | 4.111111         |
| Price Points Length         | 6.419355         | 2.888889         |

*Note: The current dataset used for model training did not include the "holiday connected" feature. The holiday connected count is computed solely for this analysis.*

**Key Observations**

- **Overall Quality:**  
  - **Similarities:**  
  Both groups have comparable totals for rows, unique weekdays, snapshot IDs, and check-in dates.

  - **Differences:**  
  The outlier group shows notably higher variability in unique rooms (9.33 vs. 5.23) and a higher holiday connected count (4.11 vs. 2.55).

- **Outlier Behavior: Two Subgroups **  
  1. **The Carlyle A Rosewood Group:**  
   - **Performance:** Final R² values are very low or negative.  
   - **Price Data:** Limited price variability (there are only unique 2 price points per data set), with many entries concentrated at one price.  
   - **Other Metrics:** Higher unique rooms and holiday connected counts suggest inconsistencies in the data for this hotel.

  2. **The New York EDITION and Park Hyatt New York Group:**  
    - **Performance:** Despite being flagged as outliers, these cases exhibit high final R² (good model performance).  
    - **Price Data:** They display greater diversity in price points and more consistent room data, which might be influenced by market factors not captured in the current model.



**Next Steps and Conclusions**

- The overall data quality appears acceptable. Although adding the holiday connected feature is an option, my intuition is that it may not significantly improve the model—and could potentially add more noise.
- The limited price variability in the Carlyle A Rosewood subgroup is a promising direction for further investigation.
- The immediate next step should be to examine the algorithm's stopping criteria. In particular, we need to understand why the active sampling stops after only 3 requests and explore the effects of modifying this behavior to request more data points before termination.



#### Why do we stop? (Minimum Sample Threshold)

**Questions**: 
- Why did the model stop sampling after 3 iterations? 
- what would happen if we set a minimum number of iterations (higher)?

##### Debugging the Stop



In this section, we investigate why the basic active sampling loop stops after only a few iterations. We focus on three parameter combinations from the suspected outlier group:
- **params_mostData[9]** (The New York EDITION)
- **params_mostData[21]** (The Carlyle A Rosewood)
- **params_mostData[29]** (Park Hyatt New York)

We use a debug version of the active sampling loop that prints:
- The training and candidate indices at each iteration.
- The uncertainty values for each candidate.
- The final training Data.
- The maximum uncertainty and the chosen candidate index.
- The current `R²` score.

This information should shed light on why the loop is stopping early, helping us determine if a higher minimum sample threshold might be necessary.


In [ ]:
def active_sampling_loop_debug_table(X, y, max_iterations=10, uncertainty_threshold=0.05, initial_sample_indices=None):
    n_samples = X.shape[0]
    if initial_sample_indices is None:
        initial_sample_indices = [0, n_samples // 2, n_samples - 1]
    
    training_indices = set(initial_sample_indices)
    candidate_indices = set(range(n_samples)) - training_indices
    iteration_log = []
    
    for iteration in range(max_iterations):
        X_train = X.iloc[sorted(list(training_indices))]
        y_train = y.iloc[sorted(list(training_indices))]

        model = initialize_model()
        model.fit(X_train, y_train)
        
        candidate_list = sorted(list(candidate_indices))
        X_candidates = X.iloc[candidate_list]
        y_pred_candidates, std_candidates = model.predict(X_candidates, return_std=True)
        
        max_std = np.max(std_candidates)
        chosen_candidate_idx = candidate_list[np.argmax(std_candidates)]
        
        y_pred_full = model.predict(X)
        current_r2 = r2_score(y, y_pred_full)
        
        iteration_details = {
            "Iteration": iteration,
            "Training Indices": ", ".join(map(str, sorted(training_indices))),
            "Candidate Indices": ", ".join(map(str, sorted(candidate_indices))),
            "Candidate Uncertainties": str(np.round(std_candidates, 4).tolist()),
            "Max Uncertainty": round(max_std, 4),
            "Chosen Candidate": chosen_candidate_idx,
            "Current R²": round(current_r2, 4)
        }
        iteration_log.append(iteration_details)
        
        if max_std < uncertainty_threshold:
            break
        
        training_indices.add(chosen_candidate_idx)
        candidate_indices.remove(chosen_candidate_idx)
    
    X_final = X.iloc[sorted(list(training_indices))]
    y_final = y.iloc[sorted(list(training_indices))]
    final_model = initialize_model()
    final_model.fit(X_final, y_final)
    final_predictions = final_model.predict(X)
    final_r2 = r2_score(y, final_predictions)
    
    final_details = {
        "Final R²": round(final_r2, 4),
        "Total Samples Used": len(training_indices)
    }
    
    print("Iteration Details:")
    display(pd.DataFrame(iteration_log))
    print("\nFinal Details:")
    display(pd.DataFrame([final_details]))
    
    print("\nFinal Training Data (first 10 rows):")
    display(X_final.head(10))
    print("Final Training Target (first 10 rows):")
    display(y_final.head(10))
    
    return {
        'final_model': final_model,
        'final_predictions': final_predictions,
        'final_r2': final_r2,
        'iteration_log': iteration_log,
        'total_samples_used': len(training_indices)
    }

# Debug the basic loop for selected parameters from the suspected outlier group.
params_to_debug = [params_mostData[9], params_mostData[21], params_mostData[29]]

for param in params_to_debug:
    print("--------------------------------------------------")
    print(f"Debugging for parameter: {param['Snapshot Date']} | {param['Hotel Name']} | {param['Discount Code']}")
    X_debug, y_debug, scaler, reverse_scaling = prepare_training_data(param, data_filtered)
    result_debug = active_sampling_loop_debug_table(X_debug, y_debug, max_iterations=15, uncertainty_threshold=0.05)
    print("--------------------------------------------------\n")


**Analysis and Improvements: Understanding R² and Model Behavior**

 **What We Found**
- **Initial Sample Homogeneity:**  
  We observed that the three initial samples (selected using indices `[0, n_samples // 2, n_samples - 1]`) in out outliers cased often turn out to be very similar. having the same discount price. 
  - This causes the model to quickly learn a prediction near initial indices discount price.

 **A Reminder About R²**
- **R² (Coefficient of Determination):**  
  Measures how well the model explains the variance in the target variable.
  - **R² ≈ 1:** The model explains nearly all the variability.
  - **R² ≈ 0:** The model performs no better than simply predicting the mean.
  - **Negative R²:** The model performs worse than predicting the mean.

 **Illustrative Examples**
1. **Example 1:** `[1, 3, 5, 7, 1, 8, 4, 7, 1]`
   - **Average Calculation:**  
     Sum = 37, Average ≈ 4.11  
   - **Scenario:**  
     If the model starts with three initial samples that are all `1`, it learns that the target is about 1.  
     However, because the overall average is 4.11, predicting 1 is far off, resulting in a very poor model fit (negative R²).

2. **Example 2:** `[1, 1, 1, 1, 1, 1, 1, 3, 1]`
   - **Average Calculation:**  
     Sum = 11, Average ≈ 1.22  
   - **Scenario:**  
     If the model again starts with three `1`s, it learns that the target is about 1.  
     Here, because the overall average is close to 1 (1.22), the prediction is nearly as good as the average, leading to an R² close to 0.

These examples illustrate that when initial samples are too similar, the model essentially learns to predict a single value—its prediction is then almost equivalent to the mean, causing R² to be very low or even negative.

 **How to Improve the Model**
**Potential Strategies:**
- **Quantile-Based (Sorted) Sampling:**  
  - *Pros:* Ensures that the initial samples cover the full range of discount prices.
  - *Cons:* Requires additional preprocessing to determine quantile indices. im most cases this would lead to a trade of with sampling amount.
- **Increasing the Initial Sample Count:**  
  - *Pros:* Captures more variability by starting with more diverse data points.
  - *Cons:* Increases computational cost in early iterations + may not solve the problem in all cases (see example 2).
- **Random or Stratified Sampling:**  
  - *Pros:* Reduces bias and ensures different segments of the data are represented.
  - *Cons:* May yield inconsistent results unless a fixed random seed is used + may not solve the problem in all cases (see example 2).

**Current Approach:**  
We currently sort the sub-dataset by discount price so that the initial samples are drawn from different parts of the distribution. However, when the data are naturally homogeneous, this method may still select very similar points.

 **Summary**
When the initial sample points are too similar, the model essentially learns to predict a value near the average, leading to an R² near 0 or even negative scores. In contrast, a more diverse set of initial samples could improve the model’s ability to capture the data’s variability. Our next steps will focus on refining the sampling strategy—possibly using quantile-based or stratified sampling—and exploring a higher minimum sample threshold to achieve better model performance.


##### 

### Sorting the data and visualizing the results

I add the sort option to `prepare_training_data`, if sort=true the data will be sorted by discount price.
```py

if sorted:
        normalized = normalized.sort_values(by="Discount Price")

```

and added the same option to `run_active_sampling_stagnant` and `run_active_sampling_basic`.

lets see the result.

In [ ]:
results_basic = run_active_sampling_basic(params_mostData, data_filtered, sort=True, verbose=False, **basic_hyperparams)
visualize_aggregated_results(results_basic, "Basic - sorted", basic_hyperparams)

In [ ]:
results_stagnant = run_active_sampling_stagnant(params_mostData, data_filtered, sort=True ,verbose=False, **stagnant_hyperparams)
visualize_aggregated_results(results_stagnant, "Stagnant - sorted", stagnant_hyperparams)

![Basic results](./images/step4-basic-sorted-Result(pre-tuning).png)
![stagnat results](./images/step4-stagnat-sorted-Result(pre-tuning).png)

**Wow!** This improvement is insane! but the data utilization is not so great, 67% percent for basic and 48% for stagnant, we can do better!
Lets fine tune the hyper parameters to lose some performance for less trining.

### Fine-tuning hyperparameters  to get the best R^2 to Data utilization

Our goal is to strike the right balance between model performance (as measured by R²) and data efficiency (minimizing the percentage of available data used).
In our active sampling loops, the stopping criteria play a key role in this trade-off.

 **What We’re Tuning**

**For the Basic Active Sampling Loop:**
- **max_iterations:** The maximum number of iterations allowed.
- **uncertainty_threshold:** The threshold below which the model’s uncertainty is considered low enough to stop sampling.

**For the Stagnant Active Sampling Loop:**
- **max_iterations:** Same as above.
- **uncertainty_threshold:** Primary stopping threshold.
- **higher_uncertainty_threshold:** A secondary threshold used when combined with stagnation.
- **r2_improvement_threshold:** The minimum improvement in R² required between iterations.
- **max_stagnant_iterations:** The maximum number of consecutive iterations with minimal R² improvement before stopping.

**How would we do that?**

We would use **BayesianOptimization** to fine-tune our hyperparameters because it efficiently explores the hyperparameter space by balancing exploration and exploitation. This method requires fewer iterations compared to exhaustive grid search and quickly converges to the best set of parameters that maximize our model's R².

Our objective function returns the final R², and we define bounds for key hyperparameters (like uncertainty thresholds, R² improvement thresholds, and the maximum stagnant iterations). The optimized parameters help improve model performance and data utilization.

#### Fine-Tuning with Bayesian Optimization


**Fine-Tuning Hyperparameters with Bayesian Optimization**

We use Bayesian Optimization to search for the best hyperparameters for our stagnant active sampling loop. Our objective function, `objective_hyperparams_stagnant/basic`, randomly selects 5 parameter combinations from `params_mostData` (with sorted data) and runs the stagnant active sampling loop on each. It then computes the average final R² and average data utilization. A penalty is applied if the average R² falls below 0.96, ensuring that only hyperparameters yielding strong performance are favored.

The optimizer then finds the hyperparameter values that maximize our objective. We save the optimized hyperparameters in a variable (`stagnant/basic_optimized_hyperparameters`) and also pickle them to disk for future use. Finally, we run the stagnant active sampling loop with these optimized parameters and visualize the results.

This process ensures we balance high R² performance with low data utilization.


In [ ]:
%%capture
%pip install bayesian-optimization

##### Basic hyperparameters optimization

In [ ]:
import random
from bayes_opt import BayesianOptimization
import pickle

def objective_hyperparams_basic(uncertainty_threshold, max_iterations):
    max_iterations = int(max_iterations)
    hyperparams = {"max_iterations": max_iterations, "uncertainty_threshold": uncertainty_threshold}

    selected_params = random.sample(params_mostData, 3)
    results_list = run_active_sampling_basic(selected_params, data_filtered, sort=True, verbose=False, **hyperparams)
    
    avg_r2 = sum(result['final_r2'] for result in results_list) / len(results_list)
    utilization_vals = [result['total_samples_used'] / result['total_data_count'] for result in results_list]
    avg_utilization = sum(utilization_vals) / len(utilization_vals)
    score = 1 - avg_utilization
    if avg_r2 < 0.96:
        penalty = (0.96 - avg_r2) * 100
        score -= penalty
    return score

pbounds_basic = {"uncertainty_threshold": (0.01, 0.1), "max_iterations": (10, 30)}

optimizer_basic = BayesianOptimization(f=objective_hyperparams_basic, pbounds=pbounds_basic, random_state=42)
optimizer_basic.maximize(init_points=5, n_iter=150)

basic_optimized_hyperparameters = {k: int(v) if k == "max_iterations" else v for k, v in optimizer_basic.max['params'].items()}

with open("basic_optimized_hyperparameters.pkl", "wb") as f:
    pickle.dump(basic_optimized_hyperparameters, f)


In [ ]:
with open("basic_optimized_hyperparameters.pkl", "rb") as f:
    basic_optimized_hyperparameters = pickle.load(f)

results_basic = run_active_sampling_basic(params_mostData, data_filtered, sort=True, verbose=False, **basic_optimized_hyperparameters)
visualize_aggregated_results(results_basic, "Basic - sorted - optimized ", basic_optimized_hyperparameters)

##### Stagnant hyperparameters optimization

In [ ]:
import random
from bayes_opt import BayesianOptimization
import pickle

def objective_hyperparams_stagnant(uncertainty_threshold, higher_uncertainty_threshold, r2_improvement_threshold, max_stagnant_iterations, max_iterations):
    max_stagnant_iterations = int(max_stagnant_iterations)
    max_iterations = int(max_iterations)
    hyperparams = {
        "max_iterations": max_iterations,
        "uncertainty_threshold": uncertainty_threshold,
        "higher_uncertainty_threshold": higher_uncertainty_threshold,
        "r2_improvement_threshold": r2_improvement_threshold,
        "max_stagnant_iterations": max_stagnant_iterations,
    }
    selected_params = random.sample(params_mostData, 3)
    results_list = run_active_sampling_stagnant(selected_params, data_filtered, sort=True, verbose=False, **hyperparams)
    avg_r2 = sum(result['final_r2'] for result in results_list) / len(results_list)
    utilization_vals = [result['total_samples_used'] / result['total_data_count'] for result in results_list]
    avg_utilization = sum(utilization_vals) / len(utilization_vals)
    score = 1 - avg_utilization
    if avg_r2 < 0.96:
        penalty = (0.96 - avg_r2) * 100
        score -= penalty
    return score

pbounds = {
    "uncertainty_threshold": (0.01, 0.1),
    "higher_uncertainty_threshold": (0.05, 0.2),
    "r2_improvement_threshold": (1e-5, 1e-3),
    "max_stagnant_iterations": (2, 5),
    "max_iterations": (10, 30),
}

optimizer = BayesianOptimization(
    f=objective_hyperparams_stagnant,
    pbounds=pbounds,
    random_state=42,
)
optimizer.maximize(init_points=5, n_iter=150)

stagnant_optimized_hyperparameters = {
    k: int(v) if k in ['max_iterations', 'max_stagnant_iterations'] else v 
    for k, v in optimizer.max['params'].items()
}

with open("stagnant_optimized_hyperparameters.pkl", "wb") as f:
    pickle.dump(stagnant_optimized_hyperparameters, f)


In [ ]:
import pickle

with open("stagnant_optimized_hyperparameters.pkl", "rb") as f:
    stagnant_optimized_hyperparameters = pickle.load(f)

results_stagnant = run_active_sampling_stagnant(params_mostData, data_filtered, sort=True ,verbose=False, **stagnant_optimized_hyperparameters)
visualize_aggregated_results(results_stagnant, "Stagnant - sorted - optimized", stagnant_optimized_hyperparameters)


##### Hyperparameters Fine-Tuning Result Analysis

After fine-tuning our hyperparameters using Bayesian Optimization, we achieved impressive improvements in data utilization without sacrificing model performance.

**Results visualized**

**Before optimization:**
![Basic results pre](./images/step4-basic-sorted-Result(pre-tuning).png)
![stagnat results pre](./images/step4-stagnat-sorted-Result(pre-tuning).png)


**After optimization:**
![Basic results post](./images/step4-optimized-basic.png)
![stagnat results post](./images/step4-optimized-stagnat.png)


**Compression**

| **Model**    | **Metric**           | **Before Tuning**               | **After Tuning**                | **Percentage Change**        |
|--------------|----------------------|---------------------------------|---------------------------------|------------------------------|
| **Stagnant** | Data Utilization     | 48.24% (~12.78 samples/run)     | 37.52% (~9.95 samples/run)         | ~25% reduction               |
|              | R² Reduction         | 0% drop                         | ~0.13% drop (<0.0015 reduction)   | ~0.13% drop (negligible)     |
| **Basic**    | Data Utilization     | 67.82% (~18 samples/run)      | 42.94% (~11.35 samples/run)         | ~36% reduction               |
|              | R² Reduction         | 0% drop                         | No visible drop                | 0% change                   |


**Summary**

We are very pleased with the final results. With the tuned hyperparameters, our stagnant model now uses only about 10 samples per run to predict the entire month’s prices with nearly 100% accuracy. This level of efficiency and performance is truly remarkable and demonstrates the strength of our approach in balancing model accuracy with data efficiency.